<a href="https://colab.research.google.com/github/zongcai66/G10_currency_risk_for_Swiss_residents/blob/main/code/interactive_google_colab_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

⬆️ *Click on the "Open in Colab" button to visualize all plots in browser.*

# **Which of the G10 currencies is the riskiest to hold for a Swiss resident?**

In [1]:
#@title Load packages
#### check python version
expected_version = ['Python 3.10.12']
current_version = !python --version

if expected_version != current_version:
    print(f"Version mismatch: This code was written for {expected_version[0]} but is running on Python {current_version[0]}")
else:
    print(f"Python version matches: {expected_version[0]}")


#### install correct package versions
#!pip install pandas==2.2.2 plotly==5.24.1 requests==2.32.3

%reset -f
import pandas as pd
import plotly.graph_objects as go
import requests
from io import StringIO
import os
from datetime import datetime
from dateutil.relativedelta import relativedelta

start_date = '2000-01-01'
end_date = '2022-12-31'
#end_date = datetime.now().strftime('%Y-%m-%d')

Python version matches: Python 3.10.12


In [3]:
#@title Load data
import pandas as pd
import ipywidgets as widgets
from IPython.display import display


df_FX = pd.read_csv("https://raw.githubusercontent.com/zongcai66/G10_currency_risk_for_Swiss_residents/refs/heads/main/data/raw/yahoo_FX.csv")


df1 = pd.read_csv("https://raw.githubusercontent.com/zongcai66/G10_currency_risk_for_Swiss_residents/main/data/raw/interest_rates.csv")
df_int = df1

# Define button actions
def set_df1(_):
    global df_int
    df_int = df1
    button_df1.style.button_color = 'lightblue'  # Highlight df1 button
    button_df2.style.button_color = None  # Remove highlight from df2

def set_df2(_):
    global df_int
    if 'df2' not in globals():
        #### FRED data #########################################################
        urls = {
            "Norway": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01NOM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "Japan": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01JPM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "UK": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01GBM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "United States": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=on&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01USM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "Germany": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01DEM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "Australia": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01AUM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "New Zealand": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01NZM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}',
            "Canada": f'https://fred.stlouisfed.org/graph/fredgraph.csv?bgcolor=%23e1e9f0&chart_type=line&drp=0&fo=open%20sans&graph_bgcolor=%23ffffff&height=450&mode=fred&recession_bars=off&txtcolor=%23444444&ts=12&tts=12&width=1320&nt=0&thu=0&trc=0&show_legend=yes&show_axis_titles=yes&show_tooltip=yes&id=IRSTCI01CAM156N&scale=left&cosd={start_date}&coed={end_date}&line_color=%234572a7&link_values=false&line_style=solid&mark_type=none&mw=3&lw=3&ost=-99999&oet=99999&mma=0&fml=a&fq=Monthly&fam=avg&fgst=lin&fgsnd=2020-02-01&line_index=1&transformation=lin&vintage_date={end_date}&revision_date={end_date}&nd={start_date}'
        }

        # Create an empty DataFrame to store all data
        df_main = pd.DataFrame()

        # Loop through the dictionary and download the data
        for country, url in urls.items():
            response = requests.get(url)
            response.raise_for_status()  # Ensure we notice bad responses

            # Read the CSV data into a temporary DataFrame
            country_data = pd.read_csv(StringIO(response.text), delimiter=',')

            # Rename the last column to 'overnight_rate'
            country_data = country_data.rename(columns={country_data.columns[-1]: 'overnight_rate_pa',
                                                        'DATE': 'date'})

            # Add a column to identify the country
            country_data['country'] = country

            # Append to the main DataFrame
            df_main = pd.concat([df_main, country_data], ignore_index=True)


        #### SNB data ##########################################################
        # Define file names
        data_file = "devkua.csv"

        # Download data
        data_url = f"https://data.snb.ch/api/cube/zimoma/data/csv/en?fromDate={start_date[:7]}&toDate={end_date[:7]}"

        # Fetch and save data
        with requests.get(data_url) as r:
            with open(data_file, 'wb') as f:
                f.write(r.content)

        # Load the data into a DataFrame
        df_swi = pd.read_csv(data_file, skiprows=3, sep=";")
        # delete temp file
        if os.path.isfile(data_file):
            os.remove(data_file)

        # data transformation
        df_swi['Date'] = pd.to_datetime(df_swi['Date'], format='%Y-%m').dt.strftime('%Y-%m-%d')
        df_swi.rename(columns={'Date': 'date', 'Value': 'overnight_rate_pa'}, inplace=True)

        df_swi = df_swi[df_swi['D0'] == 'SARON'] #df_swi = df_swi[df_swi['D0'] == '1TGT']
        df_swi['country'] = 'Switzerland'
        df_swi = df_swi[['date', 'overnight_rate_pa', 'country']]


        #### Risk bank data ####################################################
        # URL of the Excel file
        url = f'https://www.riksbank.se/en-gb/statistics/interest-rates-and-exchange-rates/search-interest-rates-and-exchange-rates/?a=M&c=Ultimo&from={start_date}&fs=3&s=g2-SECBREPOEFF&to={end_date}&d=Comma&export=csv'

        # Fetch the content of the file
        response = requests.get(url)
        response.raise_for_status()  # Ensure we notice bad responses

        # Load the Excel file directly into a pandas DataFrame
        df_swe = pd.read_csv(StringIO(response.text), delimiter=';')

        df_swe.rename(columns={df_swe.columns[0]: 'date'}, inplace=True)
        df_swe.rename(columns={df_swe.columns[-1]: 'overnight_rate_pa'}, inplace=True)
        df_swe['date'] = pd.to_datetime(df_swe['date'], format='%Y %B').dt.strftime('%Y-%m-%d')
        df_swe['overnight_rate_pa'] = df_swe['overnight_rate_pa'].str.replace(',', '.', regex=False)
        df_swe['overnight_rate_pa'] = pd.to_numeric(df_swe['overnight_rate_pa'], errors='coerce')
        df_swe = df_swe[['date', 'overnight_rate_pa']]
        df_swe['country'] = 'Sweden'


        #### combine ###########################################################
        df2 = pd.concat([df_main, df_swi, df_swe], ignore_index=True)
    df_int = df2
    button_df1.style.button_color = None  # Remove highlight from df1
    button_df2.style.button_color = 'lightblue'  # Highlight df2 button

# Create buttons
str_start = datetime.strptime(start_date, '%Y-%m-%d').strftime('%b %Y')
str_end = (datetime.strptime(end_date, '%Y-%m-%d') - relativedelta(months=1)).strftime('%b %Y')
button_df1 = widgets.Button(description="Github (Jan 2000 to Oct 2024)", layout=widgets.Layout(width='300px', height='50px'))
button_df2 = widgets.Button(description=f"Real time data ({str_start} to {str_end})", layout=widgets.Layout(width='300px', height='50px'))

# Initially highlight the df1 button
button_df1.style.button_color = 'lightblue'

# Assign actions to buttons
button_df1.on_click(set_df1)
button_df2.on_click(set_df2)

# Display buttons
label1 = widgets.HTML(value="<b>Choose the data source:</b>")
label2 = widgets.HTML(value="(default is data from Github repository)")
display(label1, button_df1, button_df2, label2)

HTML(value='<b>Choose the data source:</b>')

Button(description='Github (Jan 2000 to Oct 2024)', layout=Layout(height='50px', width='300px'), style=ButtonS…

Button(description='Real time data (Jan 2000 to Nov 2022)', layout=Layout(height='50px', width='300px'), style…

HTML(value='(default is data from Github repository)')

In [5]:
#@title Overview: Interest rates
import plotly.graph_objects as go

fig = go.Figure()

# Add traces for each country
for country in df_int['country'].unique():
    country_data = df_int[df_int['country'] == country]
    fig.add_trace(go.Scatter(
        x=country_data['date'],
        y=country_data['overnight_rate_pa'],
        mode='lines',
        name=country,
        visible=True  # All traces visible initially
    ))

# Define event ranges and annotations
events = {
    "Full Range": (df_int['date'].min(), df_int['date'].max()),
    "Dotcom Bubble": ("2000-01-01", "2002-12-31"),
    "Global Financial Crisis": ("2007-01-01", "2009-12-31"),
    "COVID-19 Pandemic": ("2019-06-30", "2020-06-30"),
    "Post-Pandemic Inflation": ("2021-10-01", "2023-12-31")
}


annotations = [
    dict(
        x=(pd.to_datetime(start) + (pd.to_datetime(end) - pd.to_datetime(start)) / 2).strftime("%Y-%m-%d"),
        y=df_int['overnight_rate_pa'].max(),  # Place annotations near the top
        text=event,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        ax=0,
        ay=-40
    )
    for event, (start, end) in events.items() if event != "Full Range"
]

# Add buttons for each event
buttons = [
    dict(
        label=event,
        method="relayout",
        args=[{
            "xaxis.range": [start, end]
        }]
    )
    for event, (start, end) in events.items()
]

# Update layout for enhancements
fig.update_layout(
    title="Overnight Rate for Each G10 Country",
    xaxis=dict(
        title="Date",
        rangeslider=dict(visible=True),
        type="date",
        range=[df['date'].min(), df['date'].max()]
    ),
    yaxis=dict(
        title="Overnight Rate (PA)",
        range=[-1.5, 9.5]
    ),
    legend_title="Country",
    template="plotly_dark",
    hovermode="x unified",
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
    width=1600,
    height=800,
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.3,
            y=1.15,
            xanchor="left",
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="lightgrey",
            font=dict(color="black")
        )],
    annotations=annotations
)

fig.show()


In [6]:
#@title Overview: IFX returns

# Create the figure using normalized values
fig = go.Figure()

# Filter for 'Adj Close' type and drop rows with missing 'value'
df_adj_close = df_FX[df_FX['type'] == 'Adj Close'].dropna(subset=['value'])

# Normalize the 'value' column by the first value for each ticker
df_adj_close['normalized_value'] = df_adj_close.groupby('ticker', group_keys=False)['value'].apply(lambda x: x / x.iloc[0])

# Exclude rows with normalized values below 0.01 (likely errors)
df_adj_close = df_adj_close[~(df_adj_close['normalized_value'] < 0.01)]

# Add a line trace for each currency pair
for ticker in df_adj_close['ticker'].unique():
    ticker_data = df_adj_close[df_adj_close['ticker'] == ticker]
    fig.add_trace(go.Scatter(
        x=ticker_data['date'],
        y=ticker_data['normalized_value'],
        mode='lines',
        name=ticker
    ))

# Define event ranges and annotations
events = {
    "Full Range": (df_adj_close['date'].min(), df_adj_close['date'].max()),
    "EUR/CHF peg at 1.20": ("2011-07-01", "2011-11-15"),
    "Termination of EUR/CHF peg": ("2014-11-01", "2015-3-31"),
    "COVID-19 Pandemic": ("2019-06-30", "2020-06-30")
}

annotations = [
    dict(
        x=(pd.to_datetime(start) + (pd.to_datetime(end) - pd.to_datetime(start)) / 2).strftime("%Y-%m-%d"),
        y=df_adj_close['normalized_value'].max(),  # Place annotations near the top
        text=event,
        showarrow=True,
        arrowhead=2,
        arrowsize=1,
        arrowwidth=2,
        ax=0,
        ay=-40
    )
    for event, (start, end) in events.items() if event != "Full Range"
]

# Add buttons for event zooming
buttons = [
    dict(
        label=event,
        method="relayout",
        args=[{
            "xaxis.range": [start, end]
        }]
    )
    for event, (start, end) in events.items()
]

# Update layout for enhancements
fig.update_layout(
    title="Normalized FX Returns: CHF-G10 Currencies",
    xaxis=dict(
        title="Date",
        rangeslider=dict(visible=True),
        type="date",
        range=[df_adj_close['date'].min(), df_adj_close['date'].max()]
    ),
    yaxis=dict(
        title="Normalized Value",
        range=[df_adj_close['normalized_value'].min() - 0.05, df_adj_close['normalized_value'].max() + 0.05]
    ),
    legend_title="Currency Pair",
    template="plotly_dark",
    hovermode="x unified",
    plot_bgcolor="black",
    paper_bgcolor="black",
    font=dict(color="white"),
    width=1600,
    height=800,
    updatemenus=[
        dict(
            type="buttons",
            showactive=True,
            buttons=buttons,
            direction="left",
            x=0.3,
            y=1.15,
            xanchor="left",
            yanchor="top",
            pad={"r": 10, "t": 10},
            bgcolor="lightgrey",
            font=dict(color="black")
        )],
    annotations=annotations
)

fig.show()
